In [1]:
# import dependencies 
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import datetime as dt
from webdriver_manager.chrome import ChromeDriverManager
import time
import numpy as np
import os
import random

In [2]:
# Install and launch the Chrome browser
executable_path = {'executable_path': '/Users/evanbruno/.wdm/drivers/chromedriver/mac64/116.0.5845.179/chromedriver'}


browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# visit nba page 
url = 'https://www.nba.com/stats/teams/boxscores/'
browser.visit(url)

In [4]:
browser.find_by_text('I Accept').click()

In [4]:
# visit nba page 
url = 'https://www.nba.com/stats/teams/boxscores/?Season=2021-22'
browser.visit(url)

In [27]:
url = 'https://www.nba.com/stats/teams/boxscores?SeasonType=Pre+Season'
browser.visit(url)

In [5]:
# url used for entering each games data
revise_url = "https://www.nba.com/game/"

In [6]:

year = 2023
prev = 2022

select = f'{prev}-{str(year)[2:]}'

for t in range(50):

    flag = ''

    html = browser.html
    test = soup(html, 'html.parser')

    end = int(test.find(class_='Crom_cromSettings__ak6Hd').find_all('option')[-1].get_text()) + 1
    # number of pages to click through
    for pages in range(2, end):

        #sleep
        time.sleep(random.randint(1, 5))

        html = browser.html
        test = soup(html, 'html.parser')

        #finds all the hyperlinks on page 
        find = test.find_all("tbody")[2].find_all("a")

        links = []
        # iterates through hyperlinks to find ones linked to game scores
        for link in find:
            if "/game/" in link.get('href'):
                links.append(link.get('href')[6:])

        # get dates to name each csv file 
        test_date = []
        names = []
        for date in find:
            if "/games?date" in date.get('href'):
                test_date.append(date.get_text())
                names.append(f"{date.get_text()[:2]}-{date.get_text()[3:5]}")

        #iterate through each game link 
        for y in range(len(links)):

            if flag == 'break':
                break

            #used to make sure don't click the same game link twice
            
            match = test.find(class_='Crom_body__UYOcU').find('tr').find_all(class_='Anchor_anchor__cSc3P')[1].get_text()
        
            
            if match.split(' ')[1] == 'vs.':
    
                continue

            #visit game link 
            browser.visit(f"{revise_url}{links[y]}")
            time.sleep(random.randint(1, 5))

            #finds box score 
            try:
                browser.find_by_id("box-score").click()
            except:
                
                try:
                
                    time.sleep(3 )
                    browser.find_by_id("box-score").click()
                except:
                    
                    #visit game link 
                    browser.visit(f"{revise_url}{links[y]}")
                    time.sleep(random.randint(5, 10))
                    
            try:
                browser.find_by_id("box-score").click()
            except:
                
                browser.find_by_id("box-score").click()

            time.sleep(random.randint(1, 3))
            html = browser.html
            test2 = soup(html, 'html.parser')

            try:
                len(test2.find(class_="StatsTable_table__Ejk5X").find('tr').find_all("th"))

            except:
                
                time.sleep(5)

                html = browser.html
                test2 = soup(html, 'html.parser')
                

            #obtain column names
            column = []
            for i in range(1, len(test2.find(class_="StatsTable_table__Ejk5X").find('tr').find_all("th"))):
                column.append(test2.find(class_="StatsTableHead_thead__omZuF").find_all("th")[i].get_text())

            #iterate through both teams box scores
            for i in range(2):
                stats = {}
                #get team name
                team_name = test2.find_all(class_="GameBoxscore_gbTableSection__zTOUg")[i].get_text().split('Click')[0]

                if i == 0:
                    opponent = test2.find_all(class_="GameBoxscore_gbTableSection__zTOUg")[1].get_text().split('Click')[0]
                else:
                    opponent = test2.find_all(class_="GameBoxscore_gbTableSection__zTOUg")[0].get_text().split('Click')[0]

                stats["Opponent"] = []
                #iterate through rows
                for t in range(len(test2.find_all("tbody", class_="StatsTableBody_tbody__uvj_P")[i].find_all("tr"))):

                    #iterate through columns to get data 
                    for x in range(1, 21):

                        # comparison used to avoid hitting errors in obtaining wrong data 
                        if test2.find_all("tbody", class_="StatsTableBody_tbody__uvj_P")[i].find_all("tr")[t].find_all("td")[0].get_text() == "TOTALS":
                            break

                        if x == 1:
                            #adds name of player to dictionary
                            name = (test2.find_all("tbody", class_="StatsTableBody_tbody__uvj_P")[i].find_all("tr")[t].find("td").find(class_="GameBoxscoreTablePlayer_gbpNameFull__cf_sn").get_text())
                            stats[name] = []

                        #append nan if player does not play  
                        if test2.find_all("tbody", class_="StatsTableBody_tbody__uvj_P")[i].find_all("tr")[t].find_all("td")[1].get_text() in ("DNP - Coach's Decision", "DNP - Injury/Illness", "NWT - Personal", "DND - Injury/Illness", "DNP - League Suspension", "DNP - Personal", "DND - Rest", "DND_LEAGUE_SUSPENSION", "NWT-Return to Competition Reconditioning", "NWT - Not With Team", "DND - Personal", "DND-Return to Competition Reconditioning", "DNP - Rest", "DND - Coach's Decision", "NWT - League Suspension", "NWT - Injury/Illness", "NWT - Rest", "NWT_RETURN_TO_COMPETITION_RECONDITIONING", "NWT - Health and Safety Protocols", "NP - Health and Safety Protocols", "DNP - Health and Safety Protocols", "DND - Health and Safety Protocols"):
                            stats[name].append(np.nan)

                        #append stats for each player 
                        else:
                            stats[name].append(test2.find_all("tbody", class_="StatsTableBody_tbody__uvj_P")[i].find_all("tr")[t].find_all("td")[x].get_text())

                # creates dataframe 
                team_data = pd.DataFrame.from_dict(stats, orient='index', columns = column)
                #creates data column
                team_data["Date"] = test_date[y]
                team_data["Opponent"] = opponent
                team_data = team_data.dropna()

                #creates float datatype for each column
                for x in ['FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%',
                   'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', '+/-']:
                    team_data[x] = team_data[x].astype(float)

                # calculates players score according to draftkings
                team_data["Score"] = ((team_data["FGM"] - team_data["3PM"]) * 2) * 1 + (team_data["3PM"] * 0.5) + team_data["3PM"] * 3 + team_data["REB"] * 1.25 + team_data["AST"] * 1.5 + team_data["STL"] * 2 + team_data["BLK"] * 2 - team_data["TO"] * 0.5 + team_data["FTM"]
                
                team_data['Match'] = match

                #creates folder for team
                mypath = f'{year}/{team_name}'
                if not os.path.isdir(mypath):
                    os.makedirs(mypath)
                # converts dataframe to csv

                #obtain list of dates
                directory = team_name
                list_dates = []
                for filename in os.listdir(directory):
                    if filename.endswith(".csv"):
                        list_dates.append(filename)



                else:
                    team_data.to_csv(f"{mypath}/{names[y]}.csv", index=True)

        if flag == 'break':
            break
            
        print('Done')

        #revisit url
        browser.visit(url)

       #click to next page
        browser.find_option_by_text(f'{pages}').first.click()
        
    year -= 1
    prev -= 1
    
    select = f'{prev}-{str(year)[2:]}'
        
    browser.find_by_xpath('//*[@id="__next"]/div[2]/div[2]/div[3]/section[1]/div/div/div[1]/label/div/select').click()
    browser.find_by_text(select).click()

Done
Done
Done
Done
Done
Done


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=120.0.6099.216)
Stacktrace:
0   chromedriver                        0x0000000100f58d28 chromedriver + 4795688
1   chromedriver                        0x0000000100f502b3 chromedriver + 4760243
2   chromedriver                        0x0000000100b2988d chromedriver + 407693
3   chromedriver                        0x0000000100b0e67e chromedriver + 296574
4   chromedriver                        0x0000000100b0e573 chromedriver + 296307
5   chromedriver                        0x0000000100b2c052 chromedriver + 417874
6   chromedriver                        0x0000000100bbb117 chromedriver + 1003799
7   chromedriver                        0x0000000100b9fa73 chromedriver + 891507
8   chromedriver                        0x0000000100b6a143 chromedriver + 672067
9   chromedriver                        0x0000000100b6b31e chrome

In [53]:
html = browser.html
test = soup(html, 'html.parser')

test.find(class_='Crom_cromSettings__ak6Hd')

In [26]:
html = browser.html
test2 = soup(html, 'html.parser')

In [27]:
len(test2.find(class_="StatsTable_table__Ejk5X"))

2

In [20]:
team_name

"Miami HeatClick on any linked stat to view the Video and/or Shot ChartPLAYERMINFGMFGAFG%3PM3PA3P%FTMFTAFT%OREBDREBREBASTSTLBLKTOPFPTS+/-Nikola JovicN. JovicF28:0561154.53560.0000.017811004153Jaime Jaquez Jr.J. Jaquez Jr.F39:0371450.02633.3000.0000811211624Bam AdebayoB. AdebayoC38:0851435.7010.05683.3347530241512Tyler HerroT. HerroG27:5171936.84850.03310006643023215Kyle LowryK. LowryG25:523650.02540.02210003364012106Kevin LoveK. Love27:534757.1010.02633.331114311031011Duncan RobinsonD. Robinson29:5851435.731225.0000.000021013130Josh RichardsonJ. Richardson20:304757.12366.7000.001110103109R.J. HamptonR. Hampton0:40000.0000.0000.00000000000Cole SwiderC. Swider0:40010.0010.0000.00000000000Jamal CainJ. Cain0:40000.0000.0000.00000000000Orlando RobinsonO. Robinson0:40000.0000.0000.00000000000Thomas BryantT. BryantDNP - Coach's DecisionTOTALS\xa0419344.1164238.1121770.6732393014382311014"

In [61]:
browser.quit()

In [11]:
try:
    browser.find_by_id("box-score").click()
    
except:
        browser.visit(f"{revise_url}{links[y]}")
        browser.find_by_id("box-score").click()

time.sleep(random.randint(2, 5))
html = browser.html
test2 = soup(html, 'html.parser')

try:
    len(test2.find(class_="StatsTable_table__Ejk5X").find('tr').find_all("th"))

except:
    
    html = browser.html
    test2 = soup(html, 'html.parser')


In [12]:
len(test2.find(class_="StatsTable_table__Ejk5X").find('tr').find_all("th"))

21

In [13]:
html = browser.html
test2 = soup(html, 'html.parser')

test2.find(class_="StatsTable_table__Ejk5X")

<table class="StatsTable_table__Ejk5X"><thead class="StatsTableHead_thead__omZuF"><tr><th>PLAYER</th><th class="GameBoxscoreTable_stat__jWIuU">MIN</th><th class="GameBoxscoreTable_stat__jWIuU">FGM</th><th class="GameBoxscoreTable_stat__jWIuU">FGA</th><th class="GameBoxscoreTable_stat__jWIuU">FG%</th><th class="GameBoxscoreTable_stat__jWIuU">3PM</th><th class="GameBoxscoreTable_stat__jWIuU">3PA</th><th class="GameBoxscoreTable_stat__jWIuU">3P%</th><th class="GameBoxscoreTable_stat__jWIuU">FTM</th><th class="GameBoxscoreTable_stat__jWIuU">FTA</th><th class="GameBoxscoreTable_stat__jWIuU">FT%</th><th class="GameBoxscoreTable_stat__jWIuU">OREB</th><th class="GameBoxscoreTable_stat__jWIuU">DREB</th><th class="GameBoxscoreTable_stat__jWIuU">REB</th><th class="GameBoxscoreTable_stat__jWIuU">AST</th><th class="GameBoxscoreTable_stat__jWIuU">STL</th><th class="GameBoxscoreTable_stat__jWIuU">BLK</th><th class="GameBoxscoreTable_stat__jWIuU">TO</th><th class="GameBoxscoreTable_stat__jWIuU">PF</th

In [8]:
flag = ''
# number of pages to click through
for pages in range(0, 20):
    
    #sleep
    time.sleep(random.randint(1, 5))
    
    html = browser.html
    test = soup(html, 'html.parser')
    
    #finds all the hyperlinks on page 
    find = test.find_all("tbody")[2].find_all("a")

    links = []
    # iterates through hyperlinks to find ones linked to game scores
    for link in find:
        if "/game/" in link.get('href'):
            links.append(link.get('href')[6:])
    
    # get dates to name each csv file 
    test_date = []
    names = []
    for date in find:
        if "/games?date" in date.get('href'):
            test_date.append(date.get_text())
            names.append(f"{date.get_text()[:2]}-{date.get_text()[3:5]}")
            
    #iterate through each game link 
    for y in range(len(links)):
        
        if flag == 'break':
            break
        
        #used to make sure don't click the same game link twice
        
        
        if (y % 2) == 0:
            print(y)
            
            #visit game link 
            browser.visit(f"{revise_url}{links[y]}")
            time.sleep(random.randint(5, 10))

            #finds box score 
            try:
                browser.find_by_id("box-score").click()
            except:
                browser.visit(f"{revise_url}{links[y]}")
                browser.find_by_id("box-score").click()
    
            time.sleep(random.randint(1, 3))
            html = browser.html
            test2 = soup(html, 'html.parser')
            
            try:
                len(test2.find(class_="StatsTable_table__Ejk5X").find('tr').find_all("th"))
                
            except:
                
                html = browser.html
                test2 = soup(html, 'html.parser')
            
            #obtain column names
            column = []
            for i in range(1, len(test2.find(class_="StatsTable_table__Ejk5X").find('tr').find_all("th"))):
                column.append(test2.find(class_="StatsTableHead_thead__omZuF").find_all("th")[i].get_text())
            
            #iterate through both teams box scores
            for i in range(2):
                stats = {}
                #get team name
                team_name = test2.find_all(class_="GameBoxscore_gbTableSection__zTOUg")[i].get_text()
                
                if i == 0:
                    opponent = test2.find_all(class_="GameBoxscore_gbTableSection__zTOUg")[1].get_text()
                else:
                    opponent = test2.find_all(class_="GameBoxscore_gbTableSection__zTOUg")[0].get_text()
    
                stats["Opponent"] = []
                #iterate through rows
                for t in range(len(test2.find_all("tbody", class_="StatsTableBody_tbody__uvj_P")[i].find_all("tr"))):
        
                    #iterate through columns to get data 
                    for x in range(1, 21):
                    
                        # comparison used to avoid hitting errors in obtaining wrong data 
                        if test2.find_all("tbody", class_="StatsTableBody_tbody__uvj_P")[i].find_all("tr")[t].find_all("td")[0].get_text() == "TOTALS":
                            break
                        
                        if x == 1:
                            #adds name of player to dictionary
                            name = (test2.find_all("tbody", class_="StatsTableBody_tbody__uvj_P")[i].find_all("tr")[t].find("td").find(class_="GameBoxscoreTablePlayer_gbpNameFull__cf_sn").get_text())
                            stats[name] = []
                        
                        #append nan if player does not play  
                        if test2.find_all("tbody", class_="StatsTableBody_tbody__uvj_P")[i].find_all("tr")[t].find_all("td")[1].get_text() in ("DNP - Coach's Decision", "DNP - Injury/Illness", "NWT - Personal", "DND - Injury/Illness", "DNP - League Suspension", "DNP - Personal", "DND - Rest", "DND_LEAGUE_SUSPENSION", "NWT-Return to Competition Reconditioning", "NWT - Not With Team", "DND - Personal", "DND-Return to Competition Reconditioning", "DNP - Rest", "DND - Coach's Decision", "NWT - League Suspension", "NWT - Injury/Illness", "NWT - Rest", "NWT_RETURN_TO_COMPETITION_RECONDITIONING", "NWT - Health and Safety Protocols", "NP - Health and Safety Protocols", "DNP - Health and Safety Protocols", "DND - Health and Safety Protocols"):
                            stats[name].append(np.nan)
                        
                        #append stats for each player 
                        else:
                            stats[name].append(test2.find_all("tbody", class_="StatsTableBody_tbody__uvj_P")[i].find_all("tr")[t].find_all("td")[x].get_text())
                            
                # creates dataframe 
                team_data = pd.DataFrame.from_dict(stats, orient='index', columns = column)
                #creates data column
                team_data["Date"] = test_date[y]
                team_data["Opponent"] = opponent
                team_data = team_data.dropna()
                
                #creates float datatype for each column
                for x in ['FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', '+/-']:
                    team_data[x] = team_data[x].astype(float)
                    
                # calculates players score according to draftkings
                team_data["Score"] = ((team_data["FGM"] - team_data["3PM"]) * 2) * 1 + (team_data["3PM"] * 0.5) + team_data["3PM"] * 3 + team_data["REB"] * 1.25 + team_data["AST"] * 1.5 + team_data["STL"] * 2 + team_data["BLK"] * 2 - team_data["TO"] * 0.5 + team_data["FTM"]
          
                #creates folder for team
                mypath = team_name
                if not os.path.isdir(mypath):
                    os.makedirs(mypath)
                # converts dataframe to csv
        
                #obtain list of dates
                directory = team_name
                list_dates = []
                for filename in os.listdir(directory):
                    if filename.endswith(".csv"):
                        list_dates.append(filename)
                        

                
                else:
                    team_data.to_csv(f"{mypath}/{names[y]}.csv", index=True)
                    
    if flag == 'break':
        break
    
    #revisit url
    browser.visit(url)
    
   #click to next page
    browser.find_option_by_text(f'{pages}').first.click()

0


AttributeError: 'NoneType' object has no attribute 'find'

In [18]:
time.sleep(random.randint(1, 3))
html = browser.html
test2 = soup(html, 'html.parser')

In [20]:
len(test2.find(class_="StatsTable_table__Ejk5X").find_all("th"))

21

In [16]:
test2.find(class_="StatsTableHead_thead__omZuF")

In [7]:
browser.visit(f"{revise_url}{links[y]}")

In [21]:
#quit browser
browser.quit()

In [ ]:
if f"{names[y]}.csv" in list_dates:
                    print("Finished")
                    browser.quit()
                    flag = 'break'
                    break

## Historical

Used to input into LSTM model

In [151]:
flag = ''
# number of pages to click through
for pages in range(47, 51):
    
    #sleep
    time.sleep(random.randint(1, 5))
    
    html = browser.html
    test = soup(html, 'html.parser')
    
    #finds all the hyperlinks on page 
    find = test.find_all("tbody")[2].find_all("a")

    links = []
    # iterates through hyperlinks to find ones linked to game scores
    for link in find:
        if "/game/" in link.get('href'):
            links.append(link.get('href')[6:])
    
    # get dates to name each csv file 
    test_date = []
    names = []
    for date in find:
        if "/games?date" in date.get('href'):
            test_date.append(date.get_text())
            names.append(f"{date.get_text()[:2]}-{date.get_text()[3:5]}")
            
    #iterate through each game link 
    for y in range(len(links)):
        
        if flag == 'break':
            break
        
        #used to make sure don't click the same game link twice
        
        
        if (y % 2) == 0:
            print(y)
            #visit game link 
            browser.visit(f"{revise_url}{links[y]}")
            time.sleep(random.randint(5, 10))
            #finds box score 
            try: 
                browser.find_by_id("box-score").click()
            except:
                browser.visit(f"{revise_url}{links[y]}")
                browser.find_by_id("box-score").click()
    
            time.sleep(random.randint(1, 3))
            html = browser.html
            test2 = soup(html, 'html.parser')
            
            #obtain column names
            column = []
            for i in range(1, len(test2.find(class_="StatsTable_table__Ejk5X").find_all("th"))):
                column.append(test2.find(class_="StatsTableHead_thead__omZuF").find_all("th")[i].get_text())
            
            #iterate through both teams box scores
            for i in range(2):
                stats = {}
                #get team name
                team_name = test2.find_all(class_="GameBoxscoreTeamHeader_gbt__b9B6g")[i].get_text()
                
                if i == 0:
                    opponent = test2.find_all(class_="GameBoxscoreTeamHeader_gbt__b9B6g")[1].get_text()
                else:
                    opponent = test2.find_all(class_="GameBoxscoreTeamHeader_gbt__b9B6g")[0].get_text()
    
                stats["Opponent"] = []
                #iterate through rows
                for t in range(len(test2.find_all("tbody", class_="StatsTableBody_tbody__uvj_P")[i].find_all("tr"))):
        
                    #iterate through columns to get data 
                    for x in range(1, 21):
                    
                        # comparison used to avoid hitting errors in obtaining wrong data 
                        if test2.find_all("tbody", class_="StatsTableBody_tbody__uvj_P")[i].find_all("tr")[t].find_all("td")[0].get_text() == "TOTALS":
                            break
                        
                        if x == 1:
                            #adds name of player to dictionary
                            name = (test2.find_all("tbody", class_="StatsTableBody_tbody__uvj_P")[i].find_all("tr")[t].find("td").find(class_="GameBoxscoreTablePlayer_gbpNameFull__cf_sn").get_text())
                            stats[name] = []
                        
                        #append nan if player does not play  
                        if test2.find_all("tbody", class_="StatsTableBody_tbody__uvj_P")[i].find_all("tr")[t].find_all("td")[1].get_text() in ("DNP - Coach's Decision", "DNP - Injury/Illness", "NWT - Personal", "DND - Injury/Illness", "DNP - League Suspension", "DNP - Personal", "DND - Rest", "DND_LEAGUE_SUSPENSION", "NWT-Return to Competition Reconditioning", "NWT - Not With Team", "DND - Personal", "DND-Return to Competition Reconditioning", "DNP - Rest", "DND - Coach's Decision", "NWT - League Suspension", "NWT - Injury/Illness", "NWT - Rest", "NWT_RETURN_TO_COMPETITION_RECONDITIONING", "NWT - Health and Safety Protocols", "NP - Health and Safety Protocols", "DNP - Health and Safety Protocols", "DND - Health and Safety Protocols"):
                            stats[name].append(np.nan)
                        
                        #append stats for each player 
                        else:
                            stats[name].append(test2.find_all("tbody", class_="StatsTableBody_tbody__uvj_P")[i].find_all("tr")[t].find_all("td")[x].get_text())
                            
                # creates dataframe 
                team_data = pd.DataFrame.from_dict(stats, orient='index', columns = column)
                #creates data column
                team_data["Date"] = test_date[y]
                team_data["Opponent"] = opponent
                team_data = team_data.dropna()
                
                #creates float datatype for each column
                for x in ['FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', '+/-']:
                    team_data[x] = team_data[x].astype(float)
                    
                # calculates players score according to draftkings
                team_data["Score"] = ((team_data["FGM"] - team_data["3PM"]) * 2) * 1 + (team_data["3PM"] * 0.5) + team_data["3PM"] * 3 + team_data["REB"] * 1.25 + team_data["AST"] * 1.5 + team_data["STL"] * 2 + team_data["BLK"] * 2 - team_data["TO"] * 0.5 + team_data["FTM"]
          
                #creates folder for team
                mypath = f"Historical/{team_name}"
                if not os.path.isdir(mypath):
                    os.makedirs(mypath)
                # converts dataframe to csv
        
                #obtain list of dates
                directory = team_name
                list_dates = []
                for filename in os.listdir(directory):
                    if filename.endswith(".csv"):
                        list_dates.append(filename)
                        
                
                
                
                else:
                    team_data.to_csv(f"{mypath}/{names[y]}.csv", index=True)
                    
    if flag == 'break':
        break
    
    #revisit url
    browser.visit(url)
    
    #click to next page
    browser.find_option_by_text(f'{pages}').first.click()

0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48


In [16]:
browser.quit()